Import everything

In [2]:
!pip install keras sklearn pandas
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

Using TensorFlow backend.


Load data

In [23]:
df = pd.read_csv('data.csv')
df.columns = ["label","text"]
x = df['text'].values
y = df['label'].values

x_train, x_test, y_train, y_test = \
 train_test_split(x, y, test_size=0.1, random_state=123)


#Next, we'll convert text data into token vectors.

tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(x)
xtrain= tokenizer.texts_to_sequences(x_train)
xtest= tokenizer.texts_to_sequences(x_test) 


#We'll apply a padding method to add zeros and set the fixed size into each vector.

maxlen=10
xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
xtest=pad_sequences(xtest,padding='post', maxlen=maxlen)


print(x_train[3])

#Excellent, very satisfied

print(xtrain[3])

Safari Is Now the Best Browser for Blocking Third-Party Tracking Yesterday’s release of Safari 13.1 introduces stricter third-party cookie blocking features that prevents websites from identifying Safari users and tracking their web activity. This means Apple’s default web browser is now the best choice for stopping third-…
[16 20  5 85 31 10 23  1 93  9]


In [24]:
embedding_dim=50
vocab_size=500
model=Sequential()
model.add(layers.Embedding(input_dim=vocab_size,
      output_dim=embedding_dim,
      input_length=maxlen))
model.add(layers.LSTM(units=50,return_sequences=True))
model.add(layers.LSTM(units=10))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 10, 50)            25000     
_________________________________________________________________
lstm_7 (LSTM)                (None, 10, 50)            20200     
_________________________________________________________________
lstm_8 (LSTM)                (None, 10)                2440      
_________________________________________________________________
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 47,737
Trainable params: 47,737
Non-trainable params: 0
__________________________________________________

In [30]:
model.fit(xtrain,y_train, epochs=10, batch_size=16, verbose=1)

loss, acc = model.evaluate(xtrain, y_train, verbose=1)
print("Training Accuracy: ", acc)

 
loss, acc = model.evaluate(xtest, y_test, verbose=1)
print("Test Accuracy: ", acc)


Epoch 1/10
208/208 [==============================] - 1s 3ms/step - loss: 0.0620 - accuracy: 0.9904
Epoch 2/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0611 - accuracy: 0.9904
Epoch 3/10
208/208 [==============================] - 1s 3ms/step - loss: 0.0495 - accuracy: 0.9904
Epoch 4/10
208/208 [==============================] - 1s 3ms/step - loss: 0.0722 - accuracy: 0.9904
Epoch 5/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0584 - accuracy: 0.9904
Epoch 6/10
208/208 [==============================] - 1s 3ms/step - loss: 0.0572 - accuracy: 0.9904
Epoch 7/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0700 - accuracy: 0.9904
Epoch 8/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0592 - accuracy: 0.9904
Epoch 9/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0675 - accuracy: 0.9904
Epoch 10/10
208/208 [==============================] - 0s 184us/step
Training Accuracy:  0.990384638

In [28]:
ypred=model.predict(xtest)

ypred[ypred>0.5]=1 
ypred[ypred<=0.5]=0 
cm = confusion_matrix(y_test, ypred)
print(cm)

result=zip(x_test, y_test, ypred)
for i in result:
 print(i)

[[18  3]
 [ 1  2]]
('Use Permissions to Keep Scammy Apps Off Your Android Google has removed yet another batch of awful Android apps from the Google Play Store for breaking the company’s terms of service. The roundup includes 101 apps belonging to a single group known as the 2NAD network which used 27 fake developer names as a fro…', 0, array([0.], dtype=float32))
("Women's FA Cup: Chelsea’s Magdalena Eriksson scores injury-time own goal against Man City Magdalena Eriksson scores an injury-time own goal to gift Manchester City victory against Chelsea in their 2019 Women's FA Cup semi-final.", 0, array([0.], dtype=float32))
("Apollo 13: Enhanced images reveal life on stricken spacecraft Enhanced images reveal life aboard Nasa's stricken Apollo 13 spacecraft in unprecedented detail.", 0, array([0.], dtype=float32))
("US announces millions in aid for resource-rich Greenland The Trump administration will also open a consulate in the territory. amid competition for Greenland's resources.", 